# 🔍 Validation Complète avec DeepChecks - Crime Dataset 2020

## 📋 Table des Matières
1. [Niveau 1 : Intégrité des Données](#niveau-1)
2. [Niveau 2 : Drift et Distribution](#niveau-2)
3. [Niveau 3 : Performance du Modèle](#niveau-3)
4. [Résumé et Recommandations](#resume)

---

## 📦 Installation des Dépendances

In [ ]:
# Installation (si nécessaire)
# !pip install deepchecks pandas scikit-learn joblib mlflow dagshub

## 📚 Imports

In [ ]:
import pandas as pd
import numpy as np
import joblib
import warnings
from datetime import datetime

# DeepChecks
from deepchecks.tabular import Dataset
from deepchecks.tabular.suites import (
    data_integrity,
    train_test_validation,
    model_evaluation
)

# DeepChecks - Checks individuels
from deepchecks.tabular.checks import (
    # Intégrité des données
    MixedNulls,
    MixedDataTypes,
    StringMismatch,
    DataDuplicates,
    ConflictingLabels,
    OutlierSampleDetection,
    FeatureFeatureCorrelation,
    FeatureLabelCorrelation,
    
    # Drift et Distribution
    TrainTestFeatureDrift,
    TrainTestLabelDrift,
    WholeDatasetDrift,
    FeatureDrift,
    LabelDrift,
    
    # Performance du modèle
    PerformanceReport,
    ConfusionMatrixReport,
    RocReport,
    SimpleModelComparison,
    CalibrationScore,
    ModelInfo
)

# Sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import classification_report, confusion_matrix

# Visualisation
import matplotlib.pyplot as plt
import seaborn as sns

warnings.filterwarnings('ignore')

print("✅ Imports réussis")

## 🗂️ Fonction de Mapping des Crimes

In [ ]:
def map_crime_group_4(desc):
    """Regroupe les crimes en 4 catégories principales"""
    if pd.isna(desc):
        return "Other / Fraud / Public Order Crime"
    desc = str(desc).upper()
    
    # Violent Crime
    if any(k in desc for k in [
        "ASSAULT", "BATTERY", "ROBBERY", "HOMICIDE",
        "MANSLAUGHTER", "KIDNAPPING", "CRIMINAL THREATS",
        "INTIMATE PARTNER", "RAPE", "SEX", "SODOMY",
        "ORAL COPULATION", "LEWD", "PORNOGRAPHY",
        "FALSE IMPRISONMENT"
    ]):
        return "Violent Crime"
    
    # Property & Theft Crime
    if any(k in desc for k in [
        "THEFT", "BURGLARY", "SHOPLIFTING",
        "VANDALISM", "ARSON", "PICKPOCKET",
        "PURSE SNATCH", "TRESPASS", "BIKE",
        "ILLEGAL DUMPING"
    ]):
        return "Property & Theft Crime"
    
    # Vehicle-Related Crime
    if any(k in desc for k in [
        "VEHICLE", "DWOC", "MOTOR VEHICLE",
        "BOAT"
    ]):
        return "Vehicle-Related Crime"
    
    return "Other / Fraud / Public Order Crime"

print("✅ Fonction de mapping définie")

## 📂 Chargement des Données

In [ ]:
# Charger les données
print("📂 Chargement des données...")
df = pd.read_csv('data/processed/crime_2020_processed.csv')

print(f"\n📊 Shape : {df.shape}")
print(f"\n📋 Colonnes disponibles :")
print(df.columns.tolist())

# Appliquer le regroupement
df['Crime_Group'] = df['Crm Cd Desc'].apply(map_crime_group_4)

print(f"\n✅ Données chargées : {len(df):,} lignes")

## 🔧 Préparation des Données

In [ ]:
# Sélection des features
feature_cols = ['Hour', 'Day_of_week', 'Month_num', 'LAT', 'LON', 'Vict Age', 'AREA']

X = df[feature_cols].copy()
y = df['Crime_Group']

# Encoder les labels
le = LabelEncoder()
y_encoded = le.fit_transform(y)

print(f"\n📊 Features : {feature_cols}")
print(f"\n🏷️ Classes détectées :")
for i, class_name in enumerate(le.classes_):
    count = (y == class_name).sum()
    print(f"   {i}. {class_name}: {count:,} ({count/len(y)*100:.2f}%)")

# Split train/test
X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, 
    test_size=0.2, 
    random_state=42, 
    stratify=y_encoded
)

print(f"\n📊 Split :")
print(f"   Train : {X_train.shape}")
print(f"   Test  : {X_test.shape}")

# Imputation
imputer = SimpleImputer(strategy='median')
X_train_imputed = pd.DataFrame(
    imputer.fit_transform(X_train),
    columns=feature_cols,
    index=X_train.index
)
X_test_imputed = pd.DataFrame(
    imputer.transform(X_test),
    columns=feature_cols,
    index=X_test.index
)

print(f"\n✅ Imputation terminée")
print(f"   NaN restants (train) : {X_train_imputed.isna().sum().sum()}")
print(f"   NaN restants (test)  : {X_test_imputed.isna().sum().sum()}")

## 📦 Chargement du Modèle

In [ ]:
# Charger le meilleur modèle (LightGBM)
model_path = 'models/lightgbm_baseline.joblib'

try:
    model = joblib.load(model_path)
    print(f"✅ Modèle chargé : {model_path}")
    print(f"   Type : {type(model).__name__}")
except FileNotFoundError:
    print(f"⚠️ Modèle non trouvé : {model_path}")
    print("   Veuillez d'abord entraîner le modèle avec :")
    print("   python src/models/train.py --model lightgbm")
    model = None

## 🗂️ Création des Datasets DeepChecks

In [ ]:
# Créer les datasets DeepChecks
train_dataset = Dataset(
    X_train_imputed,
    label=y_train,
    cat_features=[],  # Pas de features catégorielles (toutes numériques)
    features=feature_cols
)

test_dataset = Dataset(
    X_test_imputed,
    label=y_test,
    cat_features=[],
    features=feature_cols
)

print("✅ Datasets DeepChecks créés")
print(f"   Train : {len(train_dataset)} échantillons")
print(f"   Test  : {len(test_dataset)} échantillons")

---
# <a id="niveau-1"></a>📊 Niveau 1 : Intégrité des Données

Vérification de la qualité des données : valeurs manquantes, duplicatas, outliers, cohérence.

---

## 1.1 - Suite Complète d'Intégrité

In [ ]:
print("="*80)
print("🔍 NIVEAU 1 : INTÉGRITÉ DES DONNÉES")
print("="*80)

# Créer la suite d'intégrité
integrity_suite = data_integrity()

# Exécuter sur le dataset d'entraînement
print("\n📊 Exécution de la suite d'intégrité sur les données d'entraînement...")
integrity_result = integrity_suite.run(train_dataset)

# Afficher le résultat
integrity_result.show()

## 1.2 - Checks Individuels Détaillés

### 1.2.1 - Valeurs Manquantes

In [ ]:
# Check des valeurs manquantes
mixed_nulls_check = MixedNulls()
result = mixed_nulls_check.run(train_dataset)
result.show()

### 1.2.2 - Duplicatas

In [ ]:
# Check des duplicatas
duplicates_check = DataDuplicates()
result = duplicates_check.run(train_dataset)
result.show()

### 1.2.3 - Détection d'Outliers

In [ ]:
# Détection des outliers
outlier_check = OutlierSampleDetection()
result = outlier_check.run(train_dataset)
result.show()

### 1.2.4 - Corrélations Feature-Feature

In [ ]:
# Corrélations entre features
feature_correlation_check = FeatureFeatureCorrelation()
result = feature_correlation_check.run(train_dataset)
result.show()

### 1.2.5 - Corrélations Feature-Label

In [ ]:
# Corrélations feature-label
label_correlation_check = FeatureLabelCorrelation()
result = label_correlation_check.run(train_dataset)
result.show()

## 1.3 - Résumé Intégrité des Données

In [ ]:
print("\n" + "="*80)
print("📋 RÉSUMÉ - INTÉGRITÉ DES DONNÉES")
print("="*80)

# Statistiques générales
print(f"\n📊 Statistiques générales :")
print(f"   • Nombre d'échantillons : {len(X_train_imputed):,}")
print(f"   • Nombre de features    : {len(feature_cols)}")
print(f"   • Nombre de classes     : {len(le.classes_)}")
print(f"   • Valeurs manquantes    : {X_train_imputed.isna().sum().sum()}")
print(f"   • Duplicatas            : {X_train_imputed.duplicated().sum()}")

# Distribution des classes
print(f"\n🏷️ Distribution des classes (train) :")
for i, class_name in enumerate(le.classes_):
    count = (y_train == i).sum()
    percentage = count / len(y_train) * 100
    print(f"   {i}. {class_name:40} : {count:6,} ({percentage:5.2f}%)")

print("\n✅ Niveau 1 terminé !")

---
# <a id="niveau-2"></a>📈 Niveau 2 : Drift et Distribution

Comparaison des distributions entre train et test, détection de drift.

---

## 2.1 - Suite Complète Train/Test Validation

In [ ]:
print("="*80)
print("📈 NIVEAU 2 : DRIFT ET DISTRIBUTION")
print("="*80)

# Créer la suite de validation train/test
train_test_suite = train_test_validation()

# Exécuter
print("\n📊 Exécution de la validation train/test...")
train_test_result = train_test_suite.run(
    train_dataset=train_dataset,
    test_dataset=test_dataset
)

# Afficher
train_test_result.show()

## 2.2 - Drift des Features (Détaillé)

In [ ]:
# Drift des features
feature_drift_check = TrainTestFeatureDrift()
result = feature_drift_check.run(
    train_dataset=train_dataset,
    test_dataset=test_dataset
)
result.show()

## 2.3 - Drift des Labels

In [ ]:
# Drift des labels
label_drift_check = TrainTestLabelDrift()
result = label_drift_check.run(
    train_dataset=train_dataset,
    test_dataset=test_dataset
)
result.show()

## 2.4 - Analyse Statistique du Drift

In [ ]:
# Analyse statistique détaillée du drift
from scipy.stats import ks_2samp

print("\n" + "="*80)
print("📊 ANALYSE STATISTIQUE DU DRIFT (Test de Kolmogorov-Smirnov)")
print("="*80)

drift_results = []

for feature in feature_cols:
    # Test KS
    statistic, p_value = ks_2samp(
        X_train_imputed[feature], 
        X_test_imputed[feature]
    )
    
    drift_detected = p_value < 0.05
    
    drift_results.append({
        'Feature': feature,
        'KS Statistic': statistic,
        'P-Value': p_value,
        'Drift Détecté': '🔴 OUI' if drift_detected else '✅ NON'
    })

df_drift = pd.DataFrame(drift_results)
df_drift = df_drift.sort_values('KS Statistic', ascending=False)

print("\n" + df_drift.to_string(index=False))
print("\n" + "="*80)

# Compteur de drift
n_drift = df_drift['Drift Détecté'].str.contains('OUI').sum()
print(f"\n🔍 Résultat : {n_drift}/{len(feature_cols)} features avec drift détecté")

if n_drift > 0:
    print("\n⚠️ ATTENTION : Drift détecté !")
    print("   Le modèle pourrait avoir des performances dégradées.")
else:
    print("\n✅ Aucun drift significatif détecté.")
    print("   Les distributions train/test sont similaires.")

## 2.5 - Visualisations des Distributions

In [ ]:
# Visualiser les distributions train vs test
fig, axes = plt.subplots(3, 3, figsize=(15, 12))
axes = axes.ravel()

for i, feature in enumerate(feature_cols):
    ax = axes[i]
    
    # Histogrammes
    ax.hist(X_train_imputed[feature], bins=30, alpha=0.5, label='Train', density=True)
    ax.hist(X_test_imputed[feature], bins=30, alpha=0.5, label='Test', density=True)
    
    ax.set_xlabel(feature)
    ax.set_ylabel('Densité')
    ax.legend()
    ax.set_title(f'Distribution : {feature}')

# Cacher le dernier subplot vide
if len(feature_cols) < len(axes):
    for i in range(len(feature_cols), len(axes)):
        axes[i].axis('off')

plt.tight_layout()
plt.savefig('reports/deepchecks_distributions.png', dpi=150, bbox_inches='tight')
print("\n✅ Graphique sauvegardé : reports/deepchecks_distributions.png")
plt.show()

---
# <a id="niveau-3"></a>🎯 Niveau 3 : Performance du Modèle

Évaluation complète des performances du modèle sur le test set.

---

## 3.1 - Prédictions

In [ ]:
print("="*80)
print("🎯 NIVEAU 3 : PERFORMANCE DU MODÈLE")
print("="*80)

if model is not None:
    # Prédictions
    y_pred_train = model.predict(X_train_imputed)
    y_pred_test = model.predict(X_test_imputed)
    
    # Probabilités (si disponible)
    if hasattr(model, 'predict_proba'):
        y_proba_test = model.predict_proba(X_test_imputed)
    else:
        y_proba_test = None
    
    print("\n✅ Prédictions générées")
    print(f"   Train : {len(y_pred_train)} prédictions")
    print(f"   Test  : {len(y_pred_test)} prédictions")
else:
    print("\n⚠️ Aucun modèle chargé, skip Niveau 3")

## 3.2 - Suite Complète d'Évaluation

In [ ]:
if model is not None:
    # Créer la suite d'évaluation
    model_eval_suite = model_evaluation()
    
    # Exécuter
    print("\n📊 Exécution de l'évaluation du modèle...")
    model_eval_result = model_eval_suite.run(
        train_dataset=train_dataset,
        test_dataset=test_dataset,
        model=model
    )
    
    # Afficher
    model_eval_result.show()
else:
    print("⚠️ Skip suite d'évaluation (pas de modèle)")

## 3.3 - Rapport de Performance Détaillé

In [ ]:
if model is not None:
    performance_check = PerformanceReport()
    result = performance_check.run(
        train_dataset=train_dataset,
        test_dataset=test_dataset,
        model=model
    )
    result.show()
else:
    print("⚠️ Skip performance report")

## 3.4 - Matrice de Confusion

In [ ]:
if model is not None:
    confusion_matrix_check = ConfusionMatrixReport()
    result = confusion_matrix_check.run(
        train_dataset=train_dataset,
        test_dataset=test_dataset,
        model=model
    )
    result.show()
else:
    print("⚠️ Skip confusion matrix")

## 3.5 - Métriques Détaillées par Classe

In [ ]:
if model is not None:
    from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
    
    print("\n" + "="*80)
    print("📊 MÉTRIQUES DÉTAILLÉES")
    print("="*80)
    
    # Métriques globales
    train_acc = accuracy_score(y_train, y_pred_train)
    test_acc = accuracy_score(y_test, y_pred_test)
    test_f1 = f1_score(y_test, y_pred_test, average='weighted')
    test_precision = precision_score(y_test, y_pred_test, average='weighted', zero_division=0)
    test_recall = recall_score(y_test, y_pred_test, average='weighted', zero_division=0)
    
    print(f"\n🎯 Métriques Globales :")
    print(f"   • Train Accuracy    : {train_acc:.4f} ({train_acc*100:.2f}%)")
    print(f"   • Test Accuracy     : {test_acc:.4f} ({test_acc*100:.2f}%)")
    print(f"   • Test F1-Score     : {test_f1:.4f}")
    print(f"   • Test Precision    : {test_precision:.4f}")
    print(f"   • Test Recall       : {test_recall:.4f}")
    
    # Overfitting/Underfitting
    gap = train_acc - test_acc
    print(f"\n📉 Écart Train-Test : {gap:.4f} ({gap*100:.2f}%)")
    if gap > 0.1:
        print("   ⚠️ OVERFITTING détecté !")
    elif gap < 0:
        print("   ⚠️ UNDERFITTING détecté !")
    else:
        print("   ✅ Bon équilibre train/test")
    
    # Rapport par classe
    print(f"\n📋 Classification Report :")
    print(classification_report(
        y_test, 
        y_pred_test, 
        target_names=le.classes_,
        zero_division=0
    ))
else:
    print("⚠️ Skip métriques détaillées")

## 3.6 - Feature Importance

In [ ]:
if model is not None and hasattr(model, 'feature_importances_'):
    # Feature importance
    importances = model.feature_importances_
    feature_importance_df = pd.DataFrame({
        'Feature': feature_cols,
        'Importance': importances
    }).sort_values('Importance', ascending=False)
    
    print("\n" + "="*80)
    print("🌟 FEATURE IMPORTANCE")
    print("="*80)
    print(feature_importance_df.to_string(index=False))
    
    # Graphique
    plt.figure(figsize=(10, 6))
    plt.barh(feature_importance_df['Feature'], feature_importance_df['Importance'])
    plt.xlabel('Importance')
    plt.title('Feature Importance - LightGBM')
    plt.tight_layout()
    plt.savefig('reports/deepchecks_feature_importance.png', dpi=150, bbox_inches='tight')
    print("\n✅ Graphique sauvegardé : reports/deepchecks_feature_importance.png")
    plt.show()
else:
    print("⚠️ Feature importance non disponible pour ce modèle")

---
# <a id="resume"></a>📋 Résumé et Recommandations

---

In [ ]:
print("="*80)
print("📋 RÉSUMÉ GÉNÉRAL - VALIDATION DEEPCHECKS")
print("="*80)

print(f"\n📅 Date : {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"📊 Dataset : crime_2020_processed.csv")
print(f"🤖 Modèle : {type(model).__name__ if model else 'N/A'}")

print("\n" + "="*80)
print("✅ NIVEAU 1 : INTÉGRITÉ DES DONNÉES")
print("="*80)
print(f"   • Échantillons       : {len(X_train_imputed):,}")
print(f"   • Features           : {len(feature_cols)}")
print(f"   • Valeurs manquantes : {X_train_imputed.isna().sum().sum()}")
print(f"   • Duplicatas         : {X_train_imputed.duplicated().sum()}")

print("\n" + "="*80)
print("✅ NIVEAU 2 : DRIFT ET DISTRIBUTION")
print("="*80)
if 'df_drift' in locals():
    n_drift = df_drift['Drift Détecté'].str.contains('OUI').sum()
    print(f"   • Features avec drift : {n_drift}/{len(feature_cols)}")
    if n_drift > 0:
        print("   ⚠️ ATTENTION : Drift détecté sur certaines features")
        print("      → Considérer un réentraînement avec données plus récentes")
    else:
        print("   ✅ Distributions train/test similaires")

print("\n" + "="*80)
print("✅ NIVEAU 3 : PERFORMANCE DU MODÈLE")
print("="*80)
if model is not None:
    print(f"   • Test Accuracy  : {test_acc:.4f} ({test_acc*100:.2f}%)")
    print(f"   • Test F1-Score  : {test_f1:.4f}")
    print(f"   • Écart Train-Test : {gap:.4f}")
    
    if gap > 0.1:
        print("   ⚠️ Overfitting détecté")
    elif test_acc < 0.6:
        print("   ⚠️ Performance modérée (< 60%)")
    else:
        print("   ✅ Performance acceptable")
else:
    print("   ⚠️ Aucun modèle évalué")

print("\n" + "="*80)
print("💡 RECOMMANDATIONS")
print("="*80)

recommendations = []

# Recommandations basées sur les résultats
if model is not None and test_acc < 0.6:
    recommendations.append("📈 Améliorer les performances (<60%) :")
    recommendations.append("   • Ajouter plus de features (temporelles, spatiales)")
    recommendations.append("   • Optimiser les hyperparamètres (GridSearch)")
    recommendations.append("   • Combiner avec données 2021 pour plus de samples")

if 'n_drift' in locals() and n_drift > 0:
    recommendations.append("🔄 Drift détecté :")
    recommendations.append("   • Réentraîner le modèle avec données plus récentes")
    recommendations.append("   • Mettre en place monitoring continu du drift")

if model is not None and gap > 0.1:
    recommendations.append("⚖️ Overfitting détecté :")
    recommendations.append("   • Augmenter la régularisation")
    recommendations.append("   • Réduire la complexité du modèle")
    recommendations.append("   • Ajouter plus de données d'entraînement")

recommendations.append("🚀 Actions prioritaires :")
recommendations.append("   1. Promouvoir le meilleur modèle en production")
recommendations.append("   2. Mettre en place l'API et le monitoring")
recommendations.append("   3. Planifier réentraînement périodique")

for rec in recommendations:
    print(rec)

print("\n" + "="*80)
print("✅ VALIDATION DEEPCHECKS TERMINÉE")
print("="*80)

print("\n📊 Rapports sauvegardés :")
print("   • reports/deepchecks_distributions.png")
if model is not None and hasattr(model, 'feature_importances_'):
    print("   • reports/deepchecks_feature_importance.png")

print("\n💾 Pour sauvegarder les rapports HTML :")
print("   integrity_result.save_as_html('reports/deepchecks_integrity.html')")
print("   train_test_result.save_as_html('reports/deepchecks_train_test.html')")
if model is not None:
    print("   model_eval_result.save_as_html('reports/deepchecks_model_eval.html')")

## 💾 Sauvegarder les Rapports HTML

In [ ]:
# Créer le dossier reports s'il n'existe pas
import os
os.makedirs('reports', exist_ok=True)

# Sauvegarder les rapports HTML
print("💾 Sauvegarde des rapports HTML...\n")

integrity_result.save_as_html('reports/deepchecks_integrity.html')
print("   ✅ reports/deepchecks_integrity.html")

train_test_result.save_as_html('reports/deepchecks_train_test.html')
print("   ✅ reports/deepchecks_train_test.html")

if model is not None:
    model_eval_result.save_as_html('reports/deepchecks_model_eval.html')
    print("   ✅ reports/deepchecks_model_eval.html")

print("\n✅ Tous les rapports sont sauvegardés dans le dossier reports/")

---

## 🎉 Validation Terminée !

Vous pouvez maintenant :
1. Consulter les rapports HTML dans `reports/`
2. Analyser les résultats détaillés
3. Prendre des décisions sur le réentraînement
4. Promouvoir le modèle en production si satisfaisant

---